### Lecture 12

Recall: last time we coded up the extended euclidean algorithm,

xgcd(a,b)


In [1]:
def xgcd(a,b):
    """Extended GCD:
    Returns (gcd, x, y) where gcd is the greatest common divisor of a and b
    The numbers x,y are such that gcd = ax+by."""
    prevx, x = 1, 0;  prevy, y = 0, 1
    while b:
        q, r = divmod(a,b)
        x, prevx = prevx - q*x, x  
        y, prevy = prevy - q*y, y
        a, b = b, r
    return a, prevx, prevy


In [2]:
# We'll also need to find base^exponent % p today
def modexp(base, exponent, p):
  val = base  
  currentExponent = 1 

  while currentExponent <= exponent/2: 
    val = (val**2) % p
    currentExponent = currentExponent*2

  remainderExponent = exponent - currentExponent
  if remainderExponent == 0:
    return val 
  else:
    extraFactor = modexp(base, remainderExponent, p)
    return (val * extraFactor) % p

The point is that it allows us to solve the following equation
where x is unknown:

a x % b = 1

For example, how do we solve 5x % 17 = 1 for x?

Find 1, x, y = xgcd(5, 17)
This tells us that 5(x)+17(y) = 1
Therefore,  5(x) % 17 = 1.

Contrast this with solving the equation 5^x % 17 = 1 for x (the discrete log problem, which is needed to break Diffie-Hellman).




For the RSA algorithm, we need the following ingredients:

1. A way to find modular inverses (just done, through extended Euclidean algorithm)
2. If gcd(a,n) = 1 then a^phi(n) % n = 1. Here phi(n) is Euler's totient function, phi(n) is the number of integers between 1 and n coprime to 
3. Difficulty of discrete log problem, i.e. it is hard to solve m^b % n = c 
for m, even if b,n,c are known. 
(This fact was already used in Diffie-Hellman)
4. Difficulty of finding factorizing large numbers.
4. A way of generating large prime numbers (not covered in detail)

Exercises:

1. Write a function that takes two primes as input, and returns a public key.
2. Write a function that takes a message m, a public key (b,n), and returns the encrypted message. 
3. Write a function that takes an enrypted message c, the base n, a private key e, and returns the decrypted message.
4. Write a function that takes an encrypted message c, the base n, and decrypts it via brute force. 

In [ ]:
xgcd(5,17)

(1, 7, -2)

E.g. solve 17x % 65537 = 1 for x


In [ ]:
xgcd(17,65537)

(1, 30841, -8)

In [ ]:
30841*17 % 65537 # x= 30841 is the modular inverse of 17 mod 65537

1

**Fermat's little theorem:**

If p is prime, then for any a not a multiple of p, we have a^(p-1) % p = 1.

#### Euler's Totient Function

Euler's totient function is the function phi defined by 

phi(n) = number of integers in [1,n] inclusive, that are coprime to n.

(a coprime to n means gcd(a,n)=1)

Example:

phi(6):
1',2,3,4,5',6
so phi(6)=2



Exercise:
Using xgcd, write a function that computes phi(n)

We now see: 

If p is prime, phi(p)=p-1, because being prime means all the integers from 1...p-1 are coprime to p.


What about phi(5*7)? Why is it 4x6?

What are the numbers between 1 and 35 
that are coprime to 35?



What about phi(3*5)? Why is it 8?

What are the numbers between 1 and 15
that are coprime to 15?

1',2',3,4',5,6,7',8',9,10,11',12,13',14',15

If a is coprime to 15, then 
a is coprime to 3, and a is coprime to 5. 

On the other hand, if x is coprime to 3 and y is coprime to 5, then xy is coprime to 15? Not true, x=2, y=3.

(p-1)(q-1)=pq-p-(q-1)



Theorem: If p and q are prime, phi(p*q)= (p-1)(q-1).

Proof: The integers between 1 and pq which are -not- coprime to pq are of two forms:
- A multiple of p (there are q of them)
- A multiple of q (there are p of them)
- A multiple of both p and q (there are 1 of them).
So there are p+q-1 numbers -not- coprime to pq.

#### Euler's theorem:
If gcd(a,n)=1, then 

a^phi(n) % n = 1 

(If n is prime, this Fermat's little theorem. Because phi(p)=p-1).




In [ ]:
# Try to verify Fermat's little theorem 
# a^12 % 13 = 1 whenever gcd(a,13)=1.
# Try a=3 
for i in range(0,12+1):
  print(3**i % 13)

1
3
9
1
3
9
1
3
9
1
3
9
1


In [ ]:
def phi(n):
  '''Returns the number of integers between 
  1 and n inclusive that are coprime to n.'''
  count = 0
  for i in range(1,n+1):
    if xgcd(i, n)[0] == 1:
      count = count + 1
  return count 

def phi(n):
  return sum([1 if xgcd(i,n)[0]==1 
              else 0 
              for i in range(1,n+1)])

In [ ]:
phi(5*7)

24

In [ ]:
phi(13*11)  # why is  this going to be 120

120

In [ ]:
modexp(87, p-1, p)

1

### RSA Algorithm:

1. Alice pick two large primes, p,q.
And a small prime b, must be 
coprime to (p-1)(q-1).
 Publish (pq, b) as the public key.

E.g. p=13 , q = 7, b=5 Publish (pq=91, b=5).

2. Alice solves be % (p-1)(q-1)=1 for e. 

E.g. solving 5e % 72 = 1, so e=29.

3. If Bob wants to send a message m,
he sends m^b % pq.

E.g. Bob wants to send m=17.
He sends 17**5 % 91 = 75.

4. When alice receives an encrypted message c, she compuptes c**e % pq to decrypt it.

E.g. Alice sees 75. She computes 75**29 % 91  = 17 which is the original message.


Why does the decryption work?

Alice computes c^e % pq where c=m^b % pq, so 

c^e % pq = (m^b)^e % pq = m^(be) % pq

Why is the rhs equal to m?
We know that be % phi(pq) = 1.
This means that 'be = k*phi(pq)+1' for some integer k.
So m^(be) = m^(k*phi(pq))m 
= m^(phi(pq))...m^(phi(pq)) m 
= 1....1 m. 




In [ ]:
75**29 % 91

17

In [ ]:
5*29 % 72

1